# Challenge 1

In [6]:
import base64
import binascii
import collections
import string
import itertools
import more_itertools as mit
import numpy as np

In [7]:
hex_string = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [8]:
# Convert hex string to bytes
hex_bytes = bytes.fromhex(hex_string)
# Encode bytes to base64
base64_encoded = base64.b64encode(hex_bytes).decode('utf-8')

In [9]:
base64_encoded == "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"

True

# Challenge 2

In [10]:
str1 = "1c0111001f010100061a024b53535009181c"
str2 = "686974207468652062756c6c277320657965"

In [11]:
xord = bytearray()
for x, y in zip(bytes.fromhex(str1), bytes.fromhex(str2)):
  xord.append(x ^ y)

In [12]:
binascii.hexlify(xord) == b'746865206b696420646f6e277420706c6179'

True

In [13]:
xord

bytearray(b"the kid don\'t play")

# Challenge 3

In [14]:
hex_string = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
bytes_array = binascii.unhexlify(hex_string)

In [15]:
def decrypt(cipher, key):
  return bytearray([x^key for x in cipher])

def score(b: bytearray):
  # alphabet_freq = "etaoin shrdlu"[::-1]
  alphabet_freq = "etaoin shrdlcumwfgypbvkjxqzETAOINSHRDLCUMWFGYPBVKJXQZ"[::-1]
  counter = collections.Counter(b)
  counter = sorted(counter, key=counter.get)
  score = 0
  for idx, c in enumerate(counter):
    if chr(c) not in string.printable:
      return -1000
    if (pos:=alphabet_freq.find(chr(c))) != -1:
      score += pos - (idx - pos)
  return score

def attack_single_xor(b: bytearray):
  possible = []
  for k in range(1, 256):
    v = decrypt(b, k)
    s = score(v)
    possible.append((k, v, s))
  return max(possible, key=lambda x: x[2])

In [16]:
attack_single_xor(bytes_array)

(88, bytearray(b"Cooking MC\'s like a pound of bacon"), 1213)

# Challenge 4

In [17]:
with open("4.txt", "r") as f:
  data = [binascii.unhexlify(i.strip()) for i in f.readlines()]

can = []
for x in data:
  try:
    v = attack_single_xor(x)[1]
    s = score(v)
    can.append((x, v, s))
  except Exception:
    pass
max(can, key=lambda x: x[2])

(b'{ZB\x15A]TA\x15A]P\x15ETGAL\x15\\F\x15_@XE\\[R?',
 bytearray(b'Now that the party is jumping\n'),
 1321)

# Challenge 5

In [18]:
plaintext = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"

In [19]:
def encrypt(plaintext: bytearray, key: bytearray):
  zip_list = zip(plaintext, itertools.cycle(key))
  retval = bytearray()
  for c, k in zip_list:
    retval.append(c^k)
  return retval

In [20]:
ciphertext = encrypt(plaintext.encode("utf-8"), b'ICE')
binascii.hexlify(ciphertext) == b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

True

# Challenge 6

In [21]:
def hamming_distance(b1: bytearray, b2: bytearray):
  return sum([bitcount(c1^c2) for c1, c2 in zip(b1, b2)])

def bitcount(n):
  count = 0
  while n:
    n &= (n-1)
    count += 1
  return count

In [22]:
hamming_distance("this is a test".encode("utf-8"), "wokka wokka!!!".encode("utf-8"))

37

In [23]:
with open("6.txt", "r") as f:
  data = base64.b64decode(f.read())

Find KEYSIZE with the smallest Hamming distance

In [24]:
keysize_score = {}

sample_n = 4 
for keysize in range(2, 41):
  ckiter = mit.chunked(data, keysize)
  lst = [next(ckiter) for s in range(sample_n)]
  for c1, c2 in itertools.combinations(lst, 2):
    keysize_score[keysize] = keysize_score.get(keysize, 0) + hamming_distance(c1, c2)/keysize
  keysize_score[keysize] /= sample_n

possible_keysize = min(keysize_score, key=keysize_score.get)

# The last block is not a full chunk since chunk might not be divisible to the length
# So we discard it.
ckblock = np.array(list(mit.chunked(data, possible_keysize))[:-1]).T

Divide data into blocks of KEYSIZE, transpose, then find KEY

In [25]:
# key = "".join([chr(attack_single_xor(bytearray(t))[0]) for t in ckblock])
key = ""
for t in ckblock:
  b = bytearray(t)
  k, _, _ = attack_single_xor(b)
  key += chr(k)
print(key) 
print()
print(encrypt(data, key.encode("utf-8")).decode("utf-8"))
# print(bytearray(np.array(original_message).T).decode("utf-8"))

Terminator X: Bring the noise

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take

# Challenge 7

In [26]:
with open("7.txt", "r") as f:
  ciphertext = base64.b64decode(f.read())

In [27]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

In [28]:
def aes_decrypt(key:bytearray, ciphertext:bytearray):
  backend = default_backend()
  cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
  decryptor = cipher.decryptor()

  decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

  unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
  plaintext = unpadder.update(decrypted_data) + unpadder.finalize()
  return plaintext

In [29]:
print(aes_decrypt("YELLOW SUBMARINE".encode("utf-8"), ciphertext).decode("utf-8"))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


# Challenge 8

In [30]:
with open("8.txt", "r") as f:
  ciphertexts = [binascii.unhexlify(x.strip()) for x in f]

In [31]:
def detect_ecb(ct: bytearray):
  repeated = set(tuple(x) for x in list(mit.chunked(ct, 16)))
  if len(repeated) != len(ct)//16: return True
  return False

for ct in ciphertexts:
  if detect_ecb(ct): print(ct)

b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'
